<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install yfinance==0.2.59

  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.31.1
    Uninstalling protobuf-6.31.1:
      Successfully uninstalled protobuf-6.31.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.63
    Uninstalling yfinance-0.2.63:
      Successfully uninstalled yfinance-0.2.63
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-06-13 12:16:22
-------------------
qualified stocks: 91
with latest results: 29
still star stocks: 17
-------------------
Initial Investment:  1.17 C
CY Investment:  1.44 C
Reserve:  3.50 L
Current:  1.33 C
-------------------
Today PnL: -1.14 L (-0.84%)
Current PnL: -16.72 L (-11.65%)
CY Booked + Current PnL: -10.35 L (-7.21%)
-------------------
Total profit:  2.85 L
Total loss:  -19.57 L
-------------------
Total Booked + Current PnL: 16.63 L (14.27%)
Total Booked PnL: 33.35 L (28.62%)
Curr Year Booked PnL: 6.37 L (4.78%)
Prev Year Booked PnL: 26.98 L (23.15%)
Est FTT:  2.06 C
Est FTT PnL: 72.60 L (54.48%)
Deployed:  1.17 C
Current:  1.33 C
CAGR/XIRR %: 9.41%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'AvgCost', 'FTT', 'Dev%_PE', 'Conviction', 'RSP', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,LICI,827.06,983.0,-5.02,H-LC,86.81,96521.0,11334.0,4730.0,-0.70,13.30,4.90,18.85,35.0,2.40,0.74,28.51,XY25,NTT,INSURANCE
60,RELIANCE,1291.83,1526.0,4.98,X-LC,38.46,156088.0,15279.0,10239.0,-0.67,10.85,6.56,18.13,17.0,1.49,1.20,23.25,XY25,NTT,REFINERIES
82,UJJIVANSFB,52.77,53.0,52.86,M-SC,79.12,132354.0,-10125.0,10747.0,1.26,-7.11,8.12,0.44,242.0,-0.94,1.02,56.21,OX40N,NTT,BANKS
3,ADANIPORTS,1103.69,1583.0,-0.04,M-LC,48.35,100344.0,21982.0,12051.0,-1.77,28.05,12.01,43.43,77.0,1.82,0.77,34.86,XY24,NTT,MISC
76,TATAELXSI,7332.28,7332.0,-14.30,X-MC,69.23,89929.0,-12723.0,12716.0,-0.77,-12.39,14.14,-0.00,57.0,-1.00,0.69,36.70,OX40N,NTT,IT


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
13,BERGEPAINT,561.33,680.00,-16.22,H-MC,51.65,229574.0,2235.0,45823.0,-0.30,0.98,19.96,21.14,117.0,0.05,1.77,28.54,XY24,NTT,PAINTS
32,HEROMOTOCO,4311.81,6039.03,-1.43,H-MC,57.14,150640.0,-273.0,60723.0,-1.35,-0.18,40.31,40.06,100.0,-0.00,1.16,23.21,AR,ATH,AUTO
88,VOLTAS,1278.28,1929.20,-29.62,H-MC,42.86,191910.0,168.0,97471.0,-0.81,0.09,50.79,50.92,99.0,0.00,1.48,6.30,XY25,ATH,AC
56,PGHH,13388.00,18062.58,-20.27,X-MC,12.09,201300.0,480.0,69630.0,0.14,0.24,34.59,34.92,53.0,0.01,1.55,4.93,X40,ATH,FMCG
19,CIPLA,1495.00,1795.00,-17.66,H-LC,19.78,100466.0,301.0,19802.0,-0.22,0.30,19.71,20.07,29.0,0.02,0.77,8.29,X40N,BTT,PHARMA


In [11]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
0,5PAISA,564.75,565.00,152.40,H-SC,65.93,136206.0,-50726.0,50805.0,1.42,-27.14,37.30,0.04,149.0,-1.00,1.05,30.30,OX40N,NTT,FINANCE
82,UJJIVANSFB,52.77,53.00,52.86,M-SC,79.12,132354.0,-10125.0,10747.0,1.26,-7.11,8.12,0.44,242.0,-0.94,1.02,56.21,OX40N,NTT,BANKS
37,ICICIGI,1839.64,2260.25,-13.37,H-MC,45.05,142502.0,6369.0,24753.0,1.25,4.68,17.37,22.86,34.0,0.26,1.10,17.88,X40,ATH,INSURANCE
51,LTIM,5564.16,7197.33,2.44,H-LC,83.52,197316.0,-2994.0,61780.0,1.22,-1.49,31.31,29.35,36.0,-0.05,1.52,36.32,X200,ATH,IT
90,WIPRO,243.46,420.00,-5.63,M-LC,50.55,162099.0,11154.0,98297.0,0.71,7.39,60.64,72.51,68.0,0.11,1.25,11.68,XR,NTT,IT


In [12]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,SURYODAY,189.78,240.0,57.89,H-SC,81.87,112741.0,-37185.0,76856.0,-3.29,-24.80,68.17,26.46,143.0,-0.48,0.87,44.43,XR,NTT,BANKS
58,RAJESHEXPO,517.65,518.0,1791.41,M-SC,34.07,51609.0,-85568.0,85661.0,-2.93,-62.38,165.98,0.07,232.0,-1.00,0.40,28.35,OX40N,NTT,JEWELLERY
59,RELAXO,902.64,1176.0,-40.27,H-SC,27.47,54003.0,-61535.0,96525.0,-2.80,-53.26,178.74,30.28,135.0,-0.64,0.42,5.50,X40N,NTT,FOOTWEAR
7,ASIANTILES,75.41,137.0,7022.22,M-SC,98.90,78148.0,-15662.0,92277.0,-2.79,-16.70,118.08,81.67,238.0,-0.17,0.60,51.34,XR,NTT,CERAMICS
47,KANSAINER,299.63,340.0,-68.58,H-SC,7.69,219105.0,-50562.0,86897.0,-2.56,-18.75,39.66,13.47,144.0,-0.58,1.69,9.67,XY24,NTT,PAINTS


In [13]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
82,UJJIVANSFB,52.77,53.0,52.86,M-SC,79.12,132354.0,-10125.0,10747.0,1.26,-7.11,8.12,0.44,242.0,-0.94,1.02,56.21,OX40N,NTT,BANKS
74,SYMPHONY,1306.42,1306.0,-29.56,M-SC,2.20,150008.0,-21133.0,21076.0,-1.59,-12.35,14.05,-0.03,190.0,-1.00,1.16,7.81,OX40N,NTT,DURABLES
76,TATAELXSI,7332.28,7332.0,-14.30,X-MC,69.23,89929.0,-12723.0,12716.0,-0.77,-12.39,14.14,-0.00,57.0,-1.00,0.69,36.70,OX40N,NTT,IT
87,VIPIND,488.80,489.0,-933.33,H-SC,92.31,79404.0,-15912.0,15952.0,-2.29,-16.69,20.09,0.04,153.0,-1.00,0.61,56.62,OX40N,NTT,MISC
18,CERA,8421.60,8422.0,-12.41,X-SC,62.64,90519.0,-18962.0,18964.0,-0.61,-17.32,20.95,0.00,65.0,-1.00,0.70,37.00,OX40N,NTT,CERAMICS


In [14]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
42,INFY,1461.46,2275.00,-10.27,X-LC,49.45,287635.0,26034.0,119599.0,-0.11,9.95,41.58,55.67,1.0,0.22,2.22,16.66,X40,BTT,IT
78,TCS,3794.03,4998.00,-17.44,X-LC,20.88,268944.0,-26990.0,120890.0,0.40,-9.12,44.95,31.73,2.0,-0.22,2.07,7.65,X40,BTT,IT
34,HINDUNILVR,2413.81,2723.00,-17.60,X-LC,16.48,258966.0,-11381.0,46018.0,-0.88,-4.21,17.77,12.81,5.0,-0.25,2.00,7.18,XY25,NTT,FMCG
14,BRITANNIA,4983.33,6446.05,10.87,X-LC,30.77,254357.0,25124.0,42172.0,-0.72,10.96,16.58,29.35,7.0,0.60,1.96,20.86,XY25,ATH,FMCG
55,NESTLEIND,2268.60,2755.00,-12.85,X-LC,25.27,276810.0,11384.0,45535.0,-0.83,4.29,16.45,21.44,10.0,0.25,2.13,10.75,XY25,NTT,FMCG


In [15]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
3,ADANIPORTS,1103.69,1583.0,-0.04,M-LC,48.35,100344.0,21982.0,12051.0,-1.77,28.05,12.01,43.43,77.0,1.82,0.77,34.86,XY24,NTT,MISC


In [16]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
3,ADANIPORTS,1103.69,1583.00,-0.04,M-LC,48.35,100344.0,21982.0,12051.0,-1.77,28.05,12.01,43.43,77.0,1.82,0.77,34.86,XY24,NTT,MISC
17,CAMS,3643.00,5250.99,-0.67,H-SC,71.43,111602.0,9598.0,35422.0,-2.13,9.41,31.74,44.14,125.0,0.27,0.86,27.18,X40N,ATH,MISC
83,UNIONBANK,123.87,163.00,-4.91,M-LC,84.62,167423.0,26583.0,17914.0,-2.16,18.87,10.70,31.59,86.0,1.48,1.29,45.43,XY24,NTT,BANKS
90,WIPRO,243.46,420.00,-5.63,M-LC,50.55,162099.0,11154.0,98297.0,0.71,7.39,60.64,72.51,68.0,0.11,1.25,11.68,XR,NTT,IT
38,ICICIPRULI,600.83,790.00,-12.62,H-MC,46.15,141227.0,5439.0,37312.0,-1.07,4.01,26.42,31.48,119.0,0.15,1.09,16.61,X40,ATH,INSURANCE


In [17]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
89,WHIRLPOOL,1167.49,2270.00,-32.27,M-SC,60.44,197686.0,26065.0,136008.0,0.35,15.19,68.80,94.43,194.0,0.19,1.52,45.90,XR,NTT,DURABLES
90,WIPRO,243.46,420.00,-5.63,M-LC,50.55,162099.0,11154.0,98297.0,0.71,7.39,60.64,72.51,68.0,0.11,1.25,11.68,XR,NTT,IT
11,BANKINDIA,116.91,190.00,-29.44,M-MC,63.74,186313.0,6505.0,105900.0,-1.33,3.62,56.84,62.52,170.0,0.06,1.44,34.08,XR,NTT,BANKS
39,INDIAMART,2327.09,4871.06,-49.97,H-SC,56.04,130380.0,7044.0,127785.0,-1.36,5.71,98.01,109.32,122.0,0.06,1.00,28.81,AR,ATH,MISC
32,HEROMOTOCO,4311.81,6039.03,-1.43,H-MC,57.14,150640.0,-273.0,60723.0,-1.35,-0.18,40.31,40.06,100.0,-0.00,1.16,23.21,AR,ATH,AUTO


In [18]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
21,COFFEEDAY,59.14,80.00,-53.72,L-SC,96.70,68774.0,-44775.0,84826.0,0.11,-39.43,123.34,35.27,260.0,-0.53,0.53,68.33,XR,NTT,HOTELS
52,MASFIN,326.60,399.50,-23.22,H-SC,70.33,87135.0,-10845.0,32719.0,-1.22,-11.07,37.55,22.32,141.0,-0.33,0.67,26.38,XR,ATH,FINANCE
50,LICI,827.06,983.00,-5.02,H-LC,86.81,96521.0,11334.0,4730.0,-0.70,13.30,4.90,18.85,35.0,2.40,0.74,28.51,XY25,NTT,INSURANCE
3,ADANIPORTS,1103.69,1583.00,-0.04,M-LC,48.35,100344.0,21982.0,12051.0,-1.77,28.05,12.01,43.43,77.0,1.82,0.77,34.86,XY24,NTT,MISC
35,HINDZINC,514.80,744.74,38.58,H-LC,95.60,103540.0,580.0,45413.0,0.09,0.56,43.86,44.67,25.0,0.01,0.80,33.62,X5K,ATH,METALS


In [19]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
85,VALIANTORG,512.64,838.00,-393.20,H-SC,100.0,124631.0,-8655.0,93249.0,-2.27,-6.49,74.82,63.47,142.0,-0.09,0.96,108.91,XR,NTT,CHEMICALS
7,ASIANTILES,75.41,137.00,7022.22,M-SC,98.9,78148.0,-15662.0,92277.0,-2.79,-16.70,118.08,81.67,238.0,-0.17,0.60,51.34,XR,NTT,CERAMICS
46,JPPOWER,18.73,26.20,-33.95,L-SC,97.8,134572.0,-7270.0,63841.0,0.23,-5.13,47.44,39.88,258.0,-0.11,1.04,36.38,XY24,NTT,POWER
21,COFFEEDAY,59.14,80.00,-53.72,L-SC,96.7,68774.0,-44775.0,84826.0,0.11,-39.43,123.34,35.27,260.0,-0.53,0.53,68.33,XR,NTT,HOTELS
35,HINDZINC,514.80,744.74,38.58,H-LC,95.6,103540.0,580.0,45413.0,0.09,0.56,43.86,44.67,25.0,0.01,0.80,33.62,X5K,ATH,METALS


In [20]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.74
1,25,43.21
2,50,72.03


In [21]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     31.91
XY25     14.26
X40      12.64
XR       11.03
OX40N     8.53
X40N      8.47
AR        8.08
X5K       2.36
X200      1.52
SR        1.20
Name: CurrAlloc%, dtype: float64

In [22]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    19.68
X-LC    15.61
M-SC    14.40
H-LC    11.13
H-MC     9.01
M-LC     8.50
X-MC     7.68
M-MC     5.79
X-SC     2.82
L-SC     2.50
L-LC     1.12
Name: CurrAlloc%, dtype: float64

In [23]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,15.63,-3.14,37.61
IT,11.48,-11.48,68.07
BANKS,8.53,-10.08,54.45
MISC,6.56,-26.67,76.21
FINANCE,6.40,-21.23,67.88
PAINTS,5.66,-18.96,42.40
ELECTRICAL,5.17,-4.80,50.26
HEALTHCARE,4.76,-4.10,34.51
AUTO,4.66,-11.56,60.62


In [24]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2791438.0,25
XR,1130077.0,13
AR,1020600.0,9
X40,679474.0,9
XY25,538399.0,10
X40N,429292.0,8
OX40N,323391.0,11
SR,189200.0,2
X5K,96452.0,3


In [25]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,1943660.0,19
M-SC,1458222.0,18
X-LC,638436.0,9
M-MC,593111.0,5
H-MC,558743.0,7
H-LC,510579.0,10
X-MC,418816.0,6
M-LC,341735.0,6
L-SC,273275.0,4


In [26]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,,sum,count
Conviction,Criteria,,
H-SC,XY24,917299.0,6
M-SC,XY24,583462.0,5
H-SC,AR,521267.0,3
M-MC,XY24,398696.0,3
M-SC,XR,345809.0,4
X-LC,X40,320415.0,3
H-MC,XY24,226550.0,2
M-LC,XY24,224440.0,4
H-SC,XR,202824.0,3


In [27]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 27.0 seconds
